# Protein Science AI - Comprehensive Demo

This notebook demonstrates the full capabilities of the Protein Science AI system, including:

- Protein sequence analysis using language models
- Structure prediction and analysis
- Function prediction and pathway analysis
- Multi-agent workflows
- Mutation analysis
- Visualization and interpretation

## Prerequisites

Make sure you have installed the protein science AI package and its dependencies.

In [ ]:
# Install dependencies if needed
# !pip install -r ../requirements.txt

# Import required libraries
import asyncio
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Import protein science AI components
import sys
sys.path.append('..')

from protein_science import ProteinAgent
from protein_science.collaboration import AgentCoordinator
from protein_science.foundation import ProteinLanguageModel, StructurePredictor, FunctionPredictor

print("✅ Imports successful!")

## 1. Initialize the AI System

In [ ]:
# Initialize the main protein agent
print("Initializing Protein Science AI...")
agent = ProteinAgent(
    plm_model="facebook/esm2_t33_650M_UR50D",
    enable_structure_prediction=True,
    enable_function_prediction=True,
    verbose=True
)

# Initialize the multi-agent coordinator
coordinator = AgentCoordinator(main_agent=agent)

print(f"✅ Agent initialized: {agent}")
print(f"✅ Coordinator initialized with {len(coordinator.specialized_agents)} specialized agents")

## 2. Example Protein Sequences

Let's define some interesting protein sequences to analyze.

In [ ]:
# Define example proteins
proteins = {
    "p53_partial": {
        "sequence": "MEEPQSDPSVEPPLSQETFSDLWKLLPENNVLSPLPSQAMDDLMLSPDDIEQWFTEDPGPDEAPRMPEAAPPVAPAPAAPTPAAPAPAPSWPLSSSVPSQKTYQGSYGFRLGFLHSGTAKSVTCTYSPALNKMFCQLAKTCPVQLWVDSTPPPGTRVRAMAIYKQSQHMTEVVRRCPHHERCSDSDGLAPPQHLIRVEGNLRVEYLDDRNTFRHSVVVPYEPPEVGSDCTTIHYNYMCNSSCMGGMNRRPILTIITLEDSSGNLLGRNSFEVRVCACPGRDRRTEEENLRKKGEPHHELPPGSTKRALPNNTSSSPQPKKKPLDGEYFTLQIRGRERFEMFRELNEALELKDAQAGKEPGGSRAHSSHLKSKKGQSTSRHKKLMFKTEGPDSD",
        "uniprot_id": "P04637",
        "name": "p53 tumor suppressor",
        "description": "Key tumor suppressor protein involved in cell cycle regulation"
    },
    "insulin": {
        "sequence": "MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKTRREAEDLQVGQVELGGGPGAGSLQPLALEGSLQKRGIVEQCCTSICSLYQLENYCN",
        "uniprot_id": "P01308", 
        "name": "insulin",
        "description": "Hormone regulating glucose metabolism"
    },
    "hemoglobin_alpha": {
        "sequence": "MVLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHFDLSHGSAQVKGHGKKVADALTNAVAHVDDMPNALSALSDLHAHKLRVDPVNFKLLSHCLLVTLAAHLPAEFTPAVHASLDKFLASVSTVLTSKYR",
        "uniprot_id": "P69905",
        "name": "hemoglobin alpha",
        "description": "Oxygen-carrying protein in red blood cells"
    }
}

# Display protein information
for name, info in proteins.items():
    print(f"🧬 {info['name'].upper()}")
    print(f"   Length: {len(info['sequence'])} residues")
    print(f"   UniProt: {info['uniprot_id']}")
    print(f"   Description: {info['description']}")
    print()

## 3. Basic Protein Analysis

Let's start with a comprehensive analysis of the p53 protein.

In [ ]:
# Analyze p53 protein
p53_data = proteins["p53_partial"]

print(f"🔬 Analyzing {p53_data['name']}...")
print(f"Sequence length: {len(p53_data['sequence'])} residues")

# Run comprehensive analysis
p53_result = await agent.analyze_protein(
    protein_input={
        "sequence": p53_data["sequence"],
        "uniprot_id": p53_data["uniprot_id"]
    },
    analysis_type="comprehensive",
    include_structure=True,
    include_function=True
)

print(f"✅ Analysis completed in {p53_result['analysis_time_seconds']:.2f} seconds")

### 3.1 Sequence Analysis Results

In [ ]:
# Extract and display sequence analysis results
seq_analysis = p53_result['sequence_analysis']
basic_props = seq_analysis['basic_properties']

print("📊 SEQUENCE PROPERTIES")
print("=" * 30)
print(f"Length: {basic_props['length']} residues")
print(f"Molecular Weight: ~{basic_props['molecular_weight_approx']:,} Da")
print(f"Unique Residues: {basic_props['unique_residues']}/20")
print(f"Most Common AA: {basic_props['most_common_residue']}")

# Amino acid composition
if 'patterns' in seq_analysis and 'charge_distribution' in seq_analysis['patterns']:
    charge_info = seq_analysis['patterns']['charge_distribution']
    print(f"Net Charge: {charge_info['net_charge']}")
    print(f"Charge Density: {charge_info['charge_density']:.3f}")

# Known motifs
if 'patterns' in seq_analysis and 'known_motifs' in seq_analysis['patterns']:
    motifs = seq_analysis['patterns']['known_motifs']
    if motifs:
        print("\n🎯 KNOWN MOTIFS")
        print("=" * 20)
        for motif in motifs:
            print(f"• {motif['motif']}: {motif['description']}")
            print(f"  Positions: {motif['positions']}")

### 3.2 Structure Analysis Results

In [ ]:
# Display structure analysis results
if 'structure_analysis' in p53_result:
    struct_analysis = p53_result['structure_analysis']
    
    if 'error' not in struct_analysis:
        print("🏗️ STRUCTURE ANALYSIS")
        print("=" * 25)
        
        # AlphaFold data
        if 'alphafold_data' in struct_analysis:
            print("✅ AlphaFold structure available")
            
            # Confidence scores
            if 'confidence_scores' in struct_analysis:
                conf_info = struct_analysis['confidence_scores']
                if 'overall_confidence' in conf_info:
                    print(f"Overall Confidence: {conf_info['overall_confidence']}")
                if 'confidence_type' in conf_info:
                    print(f"Confidence Type: {conf_info['confidence_type']}")
        
        # Structural properties
        if 'structure_analysis' in struct_analysis:
            struct_details = struct_analysis['structure_analysis']
            
            metrics = [
                ('Atoms', ['num_atoms', 'basic_num_atoms']),
                ('Residues', ['num_residues', 'basic_num_residues']),
                ('Chains', ['num_chains', 'basic_num_chains'])
            ]
            
            for metric_name, keys in metrics:
                for key in keys:
                    if key in struct_details:
                        print(f"{metric_name}: {struct_details[key]}")
                        break
            
            # Geometric properties
            if 'radius_of_gyration' in struct_details:
                print(f"Radius of Gyration: {struct_details['radius_of_gyration']:.2f} Å")
                
    else:
        print(f"⚠️ Structure analysis failed: {struct_analysis['error']}")
else:
    print("⚠️ No structure analysis performed")

### 3.3 Function Analysis Results

In [ ]:
# Display function analysis results
if 'function_analysis' in p53_result:
    func_analysis = p53_result['function_analysis']
    
    print("⚙️ FUNCTION ANALYSIS")
    print("=" * 25)
    
    methods_used = func_analysis.get('methods_used', [])
    print(f"Methods Used: {', '.join(methods_used)}")
    
    # Show predictions from each method
    predictions = func_analysis.get('predictions', {})
    
    for method, prediction in predictions.items():
        print(f"\n📋 {method.replace('_', ' ').title()} Results:")
        
        if isinstance(prediction, dict) and 'error' not in prediction:
            # Show key information
            if 'method' in prediction:
                print(f"  Method: {prediction['method']}")
            if 'sequence_length' in prediction:
                print(f"  Sequence Length: {prediction['sequence_length']}")
            if 'composition' in prediction:
                comp = prediction['composition']
                if 'hydrophobic_fraction' in comp:
                    print(f"  Hydrophobic Fraction: {comp['hydrophobic_fraction']:.3f}")
                if 'charged_fraction' in comp:
                    print(f"  Charged Fraction: {comp['charged_fraction']:.3f}")
        else:
            print(f"  Status: {prediction.get('error', 'No detailed results')}")
    
    # Integrated prediction
    if 'integrated_prediction' in func_analysis:
        integrated = func_analysis['integrated_prediction']
        print(f"\n🎯 Integrated Prediction:")
        print(f"  Average Confidence: {integrated.get('average_confidence', 'N/A')}")
        print(f"  Integration Method: {integrated.get('integration_method', 'N/A')}")

### 3.4 AI Insights and Recommendations

In [ ]:
# Display AI-generated insights
if 'insights' in p53_result:
    insights = p53_result['insights']
    
    print("💡 AI INSIGHTS")
    print("=" * 15)
    
    # Key findings
    key_findings = insights.get('key_findings', [])
    if key_findings:
        print("Key Findings:")
        for i, finding in enumerate(key_findings, 1):
            print(f"  {i}. {finding}")
    
    # Areas of interest
    areas = insights.get('areas_of_interest', [])
    if areas:
        print("\nAreas of Interest:")
        for i, area in enumerate(areas, 1):
            print(f"  {i}. {area}")
    
    # Confidence level
    confidence = insights.get('confidence_level', 'unknown')
    print(f"\nOverall Confidence: {confidence.upper()}")

# Experimental suggestions
if 'experiment_suggestions' in p53_result:
    suggestions = p53_result['experiment_suggestions']
    
    print("\n🧪 EXPERIMENTAL SUGGESTIONS")
    print("=" * 35)
    
    # Group by priority
    priority_groups = {'high': [], 'medium': [], 'low': []}
    
    for suggestion in suggestions:
        priority = suggestion.get('priority', 'medium')
        priority_groups[priority].append(suggestion)
    
    for priority, group in priority_groups.items():
        if group:
            print(f"\n{priority.upper()} PRIORITY:")
            for i, suggestion in enumerate(group, 1):
                print(f"  {i}. {suggestion.get('experiment', 'Unknown experiment')}")
                print(f"     Type: {suggestion.get('type', 'N/A')}")
                print(f"     Rationale: {suggestion.get('rationale', 'N/A')}")
                print()

## 4. Multi-Agent Workflows

Now let's demonstrate the multi-agent collaboration system with different workflow types.

### 4.1 Drug Discovery Workflow

In [ ]:
# Execute drug discovery workflow for p53
print("🎯 Executing Drug Discovery Workflow for p53...")

drug_workflow_result = await coordinator.execute_workflow(
    workflow_type="drug_discovery",
    protein_input={
        "sequence": p53_data["sequence"],
        "uniprot_id": p53_data["uniprot_id"]
    }
)

print(f"✅ Drug discovery workflow completed")
print(f"Status: {drug_workflow_result['status']}")
print(f"Duration: {drug_workflow_result.get('duration_seconds', 0):.2f} seconds")
print(f"Workflow ID: {drug_workflow_result['workflow_id']}")

In [ ]:
# Examine drug discovery results
if 'results' in drug_workflow_result:
    drug_results = drug_workflow_result['results']
    
    print("💊 DRUG DISCOVERY RESULTS")
    print("=" * 30)
    
    # Target analysis
    if 'target_analysis' in drug_results:
        print("✅ Target protein analysis completed")
    
    # Binding sites
    if 'binding_sites' in drug_results:
        binding_info = drug_results['binding_sites']
        print(f"Binding site analysis: {binding_info.get('method', 'Unknown')}")
    
    # Virtual screening
    if 'virtual_screening' in drug_results:
        screening_info = drug_results['virtual_screening']
        print(f"Virtual screening: {screening_info.get('method', 'Unknown')}")
    
    # Optimization suggestions
    if 'optimization_suggestions' in drug_results:
        opt_suggestions = drug_results['optimization_suggestions']
        print(f"\nOptimization Strategies ({len(opt_suggestions)} suggested):")
        for i, suggestion in enumerate(opt_suggestions, 1):
            print(f"  {i}. {suggestion.get('strategy', 'Unknown')}")
            print(f"     Description: {suggestion.get('description', 'N/A')}")
            print(f"     Priority: {suggestion.get('priority', 'N/A')}")

### 4.2 Protein Engineering Workflow

In [ ]:
# Execute protein engineering workflow
print("🔧 Executing Protein Engineering Workflow...")

engineering_result = await coordinator.execute_workflow(
    workflow_type="protein_engineering",
    protein_input={
        "sequence": proteins["insulin"]["sequence"],
        "uniprot_id": proteins["insulin"]["uniprot_id"]
    }
)

print(f"✅ Protein engineering workflow completed")
print(f"Status: {engineering_result['status']}")
print(f"Duration: {engineering_result.get('duration_seconds', 0):.2f} seconds")

In [ ]:
# Examine engineering results
if 'results' in engineering_result:
    eng_results = engineering_result['results']
    
    print("🔧 PROTEIN ENGINEERING RESULTS")
    print("=" * 35)
    
    # Optimization strategies
    if 'optimization_strategies' in eng_results:
        strategies = eng_results['optimization_strategies']
        print(f"\nOptimization Strategies ({len(strategies)} suggested):")
        for i, strategy in enumerate(strategies, 1):
            print(f"  {i}. {strategy.get('strategy', 'Unknown')}")
            print(f"     Description: {strategy.get('description', 'N/A')}")
            print(f"     Priority: {strategy.get('priority', 'N/A')}")
    
    # Experimental plan
    if 'experimental_plan' in eng_results:
        exp_plan = eng_results['experimental_plan']
        print(f"\nExperimental Plan: {exp_plan.get('experimental_plan', 'N/A')}")
        
        if 'experiments' in exp_plan:
            experiments = exp_plan['experiments']
            print(f"\nRecommended Experiments ({len(experiments)}):")
            for i, exp in enumerate(experiments, 1):
                print(f"  {i}. {exp.get('type', 'Unknown')}")
                print(f"     Description: {exp.get('description', 'N/A')}")

## 5. Mutation Analysis

Let's analyze the effects of specific mutations on protein function.

In [ ]:
# Define mutations to analyze (common p53 cancer mutations)
p53_mutations = [
    "R175H",  # Hotspot mutation
    "G245S",  # Hotspot mutation  
    "R273H",  # Hotspot mutation
    "R248W",  # Hotspot mutation
    "R282W"   # Hotspot mutation
]

print(f"🧬 Analyzing {len(p53_mutations)} p53 cancer mutations...")
print("Mutations to analyze:", ", ".join(p53_mutations))

# Execute mutation analysis workflow
mutation_result = await coordinator.execute_workflow(
    workflow_type="mutation_analysis",
    protein_input={
        "sequence": p53_data["sequence"],
        "uniprot_id": p53_data["uniprot_id"]
    },
    workflow_config={
        "mutations": p53_mutations
    }
)

print(f"✅ Mutation analysis completed")
print(f"Status: {mutation_result['status']}")
print(f"Duration: {mutation_result.get('duration_seconds', 0):.2f} seconds")

In [ ]:
# Examine mutation analysis results
if 'results' in mutation_result and 'comparative_analysis' in mutation_result['results']:
    comparative = mutation_result['results']['comparative_analysis']
    
    print("🔬 MUTATION ANALYSIS RESULTS")
    print("=" * 35)
    print(f"Wild-type ID: {comparative['wildtype_id']}")
    print(f"Mutations analyzed: {comparative['mutant_count']}")
    
    # Create summary table
    if 'comparisons' in comparative:
        comparisons = comparative['comparisons']
        
        print("\nMutation Comparison Summary:")
        print("-" * 80)
        print(f"{'Mutation':<10} {'Structural':<15} {'Functional':<15} {'Stability':<15}")
        print("-" * 80)
        
        for mutation, comparison in comparisons.items():
            structural = comparison.get('structural_changes', 'Pending')[:12]
            functional = comparison.get('functional_changes', 'Pending')[:12] 
            stability = comparison.get('stability_changes', 'Pending')[:12]
            
            print(f"{mutation:<10} {structural:<15} {functional:<15} {stability:<15}")
        
        print("-" * 80)
        print("Note: Detailed mutation analysis requires additional computational resources.")
        print("This demo shows the workflow structure.")

## 6. AI Reasoning and Q&A

Let's demonstrate the AI's reasoning capabilities by asking specific questions about our analysis.

In [ ]:
# Define questions to ask about the p53 analysis
questions = [
    "What can you tell me about the function of this protein?",
    "Is this protein likely to be stable?",
    "What experiments would you recommend to validate the structure?",
    "How confident are you in the structure prediction?",
    "What are the most interesting features of this protein?",
    "What therapeutic approaches might target this protein?"
]

print("🤖 AI REASONING AND Q&A")
print("=" * 30)
print(f"Asking {len(questions)} questions about the p53 analysis...\n")

# Ask each question and get AI responses
qa_results = []

for i, question in enumerate(questions, 1):
    print(f"❓ Question {i}: {question}")
    
    # Get AI reasoning response
    answer = agent.reason_about_results(p53_result, question)
    
    print(f"🤖 Answer: {answer}")
    print("-" * 60)
    
    qa_results.append({
        "question": question,
        "answer": answer
    })

print(f"\n✅ Q&A session completed with {len(qa_results)} responses")

## 7. Comparative Analysis

Let's compare multiple proteins to understand their similarities and differences.

In [ ]:
# Analyze all proteins for comparison
print("🔍 Comparative Analysis of Multiple Proteins")
print("=" * 50)

comparative_results = {}

for protein_name, protein_data in proteins.items():
    if protein_name != "p53_partial":  # We already analyzed p53
        print(f"\nAnalyzing {protein_data['name']}...")
        
        result = await agent.analyze_protein(
            protein_input={
                "sequence": protein_data["sequence"],
                "uniprot_id": protein_data["uniprot_id"]
            },
            analysis_type="quick",  # Use quick analysis for comparison
            include_structure=False,  # Skip structure for speed
            include_function=False    # Skip function for speed
        )
        
        comparative_results[protein_name] = result
        print(f"✅ {protein_data['name']} analysis complete")

# Add p53 results
comparative_results["p53_partial"] = p53_result

print(f"\n✅ Comparative analysis completed for {len(comparative_results)} proteins")

In [ ]:
# Create comparison table
comparison_data = []

for protein_name, result in comparative_results.items():
    protein_info = proteins[protein_name]
    seq_analysis = result['sequence_analysis']
    basic_props = seq_analysis['basic_properties']
    
    # Extract charge information if available
    net_charge = 0
    if 'patterns' in seq_analysis and 'charge_distribution' in seq_analysis['patterns']:
        net_charge = seq_analysis['patterns']['charge_distribution']['net_charge']
    
    comparison_data.append({
        'Protein': protein_info['name'],
        'UniProt': protein_info['uniprot_id'],
        'Length': basic_props['length'],
        'MW (Da)': basic_props['molecular_weight_approx'],
        'Unique AAs': basic_props['unique_residues'],
        'Most Common AA': basic_props['most_common_residue'],
        'Net Charge': net_charge,
        'Analysis Time (s)': result.get('analysis_time_seconds', 0)
    })

# Create DataFrame
comparison_df = pd.DataFrame(comparison_data)

print("📊 PROTEIN COMPARISON TABLE")
print("=" * 40)
print(comparison_df.to_string(index=False))

# Save comparison results
comparison_df.to_csv('protein_comparison.csv', index=False)
print("\n💾 Comparison table saved as 'protein_comparison.csv'")

## 8. Visualization

Let's create some visualizations of our analysis results.

In [ ]:
# Set up plotting
plt.style.use('seaborn-v0_8')
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Protein Science AI - Analysis Results', fontsize=16, fontweight='bold')

# Plot 1: Protein length comparison
ax1 = axes[0, 0]
proteins_names = comparison_df['Protein']
lengths = comparison_df['Length']
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']

bars1 = ax1.bar(proteins_names, lengths, color=colors)
ax1.set_title('Protein Sequence Lengths', fontweight='bold')
ax1.set_ylabel('Length (residues)')
ax1.tick_params(axis='x', rotation=45)

# Add value labels on bars
for bar in bars1:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 5,
             f'{int(height)}', ha='center', va='bottom')

# Plot 2: Molecular weight comparison
ax2 = axes[0, 1]
molecular_weights = comparison_df['MW (Da)'] / 1000  # Convert to kDa

bars2 = ax2.bar(proteins_names, molecular_weights, color=colors)
ax2.set_title('Molecular Weight Comparison', fontweight='bold')
ax2.set_ylabel('Molecular Weight (kDa)')
ax2.tick_params(axis='x', rotation=45)

# Add value labels
for bar in bars2:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 0.5,
             f'{height:.1f}', ha='center', va='bottom')

# Plot 3: Amino acid diversity
ax3 = axes[1, 0]
unique_aas = comparison_df['Unique AAs']

bars3 = ax3.bar(proteins_names, unique_aas, color=colors)
ax3.set_title('Amino Acid Diversity', fontweight='bold')
ax3.set_ylabel('Unique Amino Acids')
ax3.set_ylim(0, 20)
ax3.tick_params(axis='x', rotation=45)

# Add value labels
for bar in bars3:
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height + 0.2,
             f'{int(height)}', ha='center', va='bottom')

# Plot 4: Net charge comparison
ax4 = axes[1, 1]
net_charges = comparison_df['Net Charge']

# Color bars based on charge (positive=blue, negative=red, neutral=gray)
bar_colors = ['red' if charge < 0 else 'blue' if charge > 0 else 'gray' 
              for charge in net_charges]

bars4 = ax4.bar(proteins_names, net_charges, color=bar_colors)
ax4.set_title('Net Charge Comparison', fontweight='bold')
ax4.set_ylabel('Net Charge')
ax4.axhline(y=0, color='black', linestyle='-', alpha=0.3)
ax4.tick_params(axis='x', rotation=45)

# Add value labels
for bar in bars4:
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height + (0.5 if height >= 0 else -0.5),
             f'{int(height)}', ha='center', va='bottom' if height >= 0 else 'top')

plt.tight_layout()
plt.savefig('protein_analysis_visualization.png', dpi=300, bbox_inches='tight')
plt.show()

print("📊 Visualization saved as 'protein_analysis_visualization.png'")

## 9. Export and Summary

Let's create a comprehensive summary and export our results.

In [ ]:
# Create comprehensive summary
summary = {
    "analysis_summary": {
        "timestamp": datetime.now().isoformat(),
        "proteins_analyzed": len(comparative_results),
        "workflows_executed": 3,  # drug discovery, protein engineering, mutation analysis
        "mutations_analyzed": len(p53_mutations),
        "total_analysis_time": sum(result.get('analysis_time_seconds', 0) 
                                 for result in comparative_results.values())
    },
    "protein_results": comparative_results,
    "workflow_results": {
        "drug_discovery": drug_workflow_result,
        "protein_engineering": engineering_result,
        "mutation_analysis": mutation_result
    },
    "qa_results": qa_results,
    "comparison_table": comparison_df.to_dict('records')
}

# Save comprehensive results
with open('protein_science_ai_demo_results.json', 'w') as f:
    json.dump(summary, f, indent=2, default=str)

print("📁 ANALYSIS SUMMARY")
print("=" * 25)
print(f"Timestamp: {summary['analysis_summary']['timestamp']}")
print(f"Proteins Analyzed: {summary['analysis_summary']['proteins_analyzed']}")
print(f"Workflows Executed: {summary['analysis_summary']['workflows_executed']}")
print(f"Mutations Analyzed: {summary['analysis_summary']['mutations_analyzed']}")
print(f"Total Analysis Time: {summary['analysis_summary']['total_analysis_time']:.2f} seconds")

print("\n💾 EXPORTED FILES")
print("=" * 20)
print("• protein_science_ai_demo_results.json - Complete analysis results")
print("• protein_comparison.csv - Protein comparison table")
print("• protein_analysis_visualization.png - Analysis visualizations")

print("\n✅ Demo completed successfully!")
print("\nNext steps:")
print("• Try with your own protein sequences")
print("• Explore different analysis types and workflows")
print("• Use the REST API or Streamlit interface")
print("• Customize the analysis parameters for your research")

## 10. System Information

Display information about the AI system configuration and capabilities.

In [ ]:
# Display system information
print("🔧 SYSTEM INFORMATION")
print("=" * 25)

print(f"Main Agent: {agent}")
print(f"Coordinator: {coordinator}")
print(f"Protein Language Model: {agent.plm.model_name}")
print(f"Available Tools: {len(agent.tools)}")

print("\nTool Capabilities:")
for tool in agent.tools:
    print(f"• {tool['name']}: {tool['description']}")

print(f"\nSpecialized Agents: {len(coordinator.specialized_agents)}")
for agent_name in coordinator.specialized_agents.keys():
    print(f"• {agent_name.replace('_', ' ').title()} Agent")

print("\n🚀 CAPABILITIES")
print("=" * 20)
capabilities = [
    "Protein sequence analysis using language models",
    "Structure prediction and analysis", 
    "Function prediction and pathway analysis",
    "Multi-agent workflow coordination",
    "Mutation effect analysis",
    "AI reasoning and question answering",
    "Comparative protein analysis",
    "Experimental design suggestions",
    "REST API and web interfaces",
    "Extensible architecture for new tools"
]

for i, capability in enumerate(capabilities, 1):
    print(f"{i:2d}. {capability}")

print("\n🎯 Thank you for exploring Protein Science AI!")
print("   Visit the repository for more examples and documentation.")